In [1]:
import pandas as pd
from utils import connection as con
from ETL import extract as ex
from ETL import load 
from ETL import transform as tm

In [2]:
conn_db_worldcup = con.connect_world_cup()
conn_dw_worldcup = con.connect_dw_world_cup()

In [3]:
df_country = ex.extract_table("Country", conn_db_worldcup)
df_cup = ex.extract_table("Cup", conn_db_worldcup)
df_team = ex.extract_table("Team", conn_db_worldcup)
df_state = ex.extract_table("State", conn_db_worldcup)
df_stage = ex.extract_table("Stage", conn_db_worldcup)
df_day = ex.extract_table("Day", conn_db_worldcup)
df_game = ex.extract_table("Game", conn_db_worldcup)
df_details = ex.extract_table("Details", conn_db_worldcup)

# Dimensión Team

In [4]:
load.load_dw_worldcup('Team', conn_dw_worldcup, df_team)

(MySQLdb.IntegrityError) (1062, "Duplicate entry '1' for key 'Team.PRIMARY'")
[SQL: INSERT INTO `Team` (`idTeam`, `Name`) VALUES (%s, %s)]
[parameters: [(1, 'Algeria'), (2, 'Angola'), (3, 'Argentina'), (4, 'Australia'), (5, 'Austria'), (6, 'Belgium'), (7, 'Bolivia'), (8, 'Bosnia and Herzegovina')  ... displaying 10 of 84 total bound parameter sets ...  (83, 'Yugoslavia'), (84, 'Zaire')]]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


# Dimensión State

In [5]:
load.load_dw_worldcup('State', conn_dw_worldcup, df_state)

(MySQLdb.IntegrityError) (1062, "Duplicate entry '1' for key 'State.PRIMARY'")
[SQL: INSERT INTO `State` (`idState`, `State`, `Description`) VALUES (%s, %s, %s)]
[parameters: [(1, 'W', 'Win'), (2, 'D', 'Draw'), (3, 'L', 'Lose')]]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


# Dimensión Game

In [16]:
query = """
select Game.idGame, Stage.Stage, Day.Day from Game 
inner join Stage on Game.Stage_idStage = Stage.idStage
inner join Day on Game.Day_idDay = Day.idDay
"""

dw_game = tm.transform_query(query, conn_db_worldcup)
dw_game

,idGame,Stage,Day
0,1,Group A,Sunday
1,2,Group D,Sunday
2,3,Group B,Monday
3,4,Group C,Monday
4,5,Group A,Tuesday
...,...,...,...
895,896,Quarterfinals,Saturday
896,897,Semifinals,Tuesday
897,898,Semifinals,Wednesday
898,899,Third place,Saturday


In [17]:
load.load_dw_worldcup('Game', conn_dw_worldcup, dw_game)

(MySQLdb.IntegrityError) (1062, "Duplicate entry '1' for key 'Game.PRIMARY'")
[SQL: INSERT INTO `Game` (`idGame`, `Stage`, `Day`) VALUES (%s, %s, %s)]
[parameters: [(1, 'Group A', 'Sunday'), (2, 'Group D', 'Sunday'), (3, 'Group B', 'Monday'), (4, 'Group C', 'Monday'), (5, 'Group A', 'Tuesday'), (6, 'Group A', 'Wednesday'), (7, 'Group B', 'Thursday'), (8, 'Group D', 'Thursday')  ... displaying 10 of 900 total bound parameter sets ...  (899, 'Third place', 'Saturday'), (900, 'Final', 'Sunday')]]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


# Dimensión Site

In [8]:
query = """
select Cup.idCup, Cup.Year, Country.Country from Cup 
inner join Country on Cup.Country_idCountry = Country.idCountry
order by Cup.idCup ASC
-- inner join Day on Game.Day_idDay = Day.idDay
"""

dw_site = tm.transform_query(query, conn_db_worldcup)
dw_site

,idCup,Year,Country
0,1,1930,Uruguay
1,2,1934,Italy
2,3,1938,France
3,4,1950,Brazil
4,5,1954,Switzerland
5,6,1958,Sweden
6,7,1962,Chile
7,8,1966,England
8,9,1970,Mexico
9,10,1974,Germany


In [9]:
dw_site = tm.columname(dw_site,'idCup','idSite')
dw_site

,idSite,Year,Country
0,1,1930,Uruguay
1,2,1934,Italy
2,3,1938,France
3,4,1950,Brazil
4,5,1954,Switzerland
5,6,1958,Sweden
6,7,1962,Chile
7,8,1966,England
8,9,1970,Mexico
9,10,1974,Germany


In [10]:
load.load_dw_worldcup('Site', conn_dw_worldcup, dw_site)

(MySQLdb.IntegrityError) (1062, "Duplicate entry '1' for key 'Site.PRIMARY'")
[SQL: INSERT INTO `Site` (`idSite`, `Year`, `Country`) VALUES (%s, %s, %s)]
[parameters: [(1, '1930', 'Uruguay'), (2, '1934', 'Italy'), (3, '1938', 'France'), (4, '1950', 'Brazil'), (5, '1954', 'Switzerland'), (6, '1958', 'Sweden'), (7, '1962', 'Chile'), (8, '1966', 'England')  ... displaying 10 of 21 total bound parameter sets ...  (20, '2014', 'Brazil'), (21, '2018', 'Russia')]]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


# Hecho Event

In [11]:
dw_event = tm.columname(df_details,'Cup_idCup','Site_idSite')
dw_event = tm.columname(dw_event,'idDetails','idEvent')
dw_event

,idEvent,Team_idTeam,Score,State_idState,Site_idSite,Game_idGame
0,1,29,4,1,1,1
1,2,3,1,1,1,5
2,3,13,3,1,1,6
3,4,80,1,1,1,9
4,5,3,6,1,1,10
...,...,...,...,...,...,...
1795,1796,63,2,2,21,867
1796,1797,46,2,2,21,871
1797,1798,57,1,2,21,872
1798,1799,29,0,2,21,874


In [12]:
load.load_dw_worldcup('Event', conn_dw_worldcup, dw_event)

In [4]:
df = pd.read_sql("SELECT * FROM Game", con = conn_dw_worldcup)
df

,idGame,Stage,Day
0,1,Group A,Sunday
1,2,Group D,Sunday
2,3,Group B,Monday
3,4,Group C,Monday
4,5,Group A,Tuesday
...,...,...,...
895,896,Quarterfinals,Saturday
896,897,Semifinals,Tuesday
897,898,Semifinals,Wednesday
898,899,Third place,Saturday
